# Sarah

In [ ]:
# importing libraries
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.dummy import DummyRegressor
import pandas as pd
import html
import time
import csv
# from scrapy.downloadermiddlewares.retry import RetryMiddleware
# from scrapy_random_proxies.middlewares import RandomProxyMiddleware
import requests
from bs4 import BeautifulSoup
import random

import os

In [ ]:
preprocessed_top_friends_solved_problems_df.csv = pd.read_csv('preprocessed_top_friends_solved_problems_df.csv')
preprocessed_top_friends_solved_problems_df

In [ ]:
# Split the DataFrame into four equal-sized DataFrames
split_data = np.array_split(preprocessed_top_friends_solved_problems_df, 8)

# Access the individual DataFrames
preprocessed_top_friends_solved_problems_df1, preprocessed_top_friends_solved_problems_df2, preprocessed_top_friends_solved_problems_df3, preprocessed_top_friends_solved_problems_df4, preprocessed_top_friends_solved_problems_df5, preprocessed_top_friends_solved_problems_df6, preprocessed_top_friends_solved_problems_df7, preprocessed_top_friends_solved_problems_df8 = split_data[0], split_data[1], split_data[2], split_data[3], split_data[4], split_data[5], split_data[6], split_data[7]

# Check the sizes of the DataFrames
print(len(preprocessed_top_friends_solved_problems_df1)) 
print(len(preprocessed_top_friends_solved_problems_df2)) 
print(len(preprocessed_top_friends_solved_problems_df3))  
print(len(preprocessed_top_friends_solved_problems_df4))  
print(len(preprocessed_top_friends_solved_problems_df5))  
print(len(preprocessed_top_friends_solved_problems_df6))  
print(len(preprocessed_top_friends_solved_problems_df7))  
print(len(preprocessed_top_friends_solved_problems_df8))  

In [ ]:
if os.path.exists('top_friends_failed_submission.csv'):
    failed_submission = pd.read_csv("top_friends_failed_submission.csv")
    
    # Find the index where starting_point['#'] is equal to the specific value
    start_index = preprocessed_top_friends_solved_problems_df6[preprocessed_top_friends_solved_problems_df6['#'] == failed_submission['#'][0]].index[0]
    
    # Slice the DataFrame to include rows starting from the identified index
    starting_point = preprocessed_top_friends_solved_problems_df6.loc[start_index:]
    os.remove('top_friends_failed_submission.csv')

else:
    # Slice the DataFrame to include rows starting from the identified index
    starting_point = preprocessed_top_friends_solved_problems_df6
    

# Iterate over the values in 'Number' and '#' columns
for num, submission in zip(starting_point['Number'], starting_point['#']):
        # The base URL for the submission page
        submission_url = f'https://codeforces.com/contest/{num}/submission/{submission}'
        print(submission_url)

        # Send a GET request to the URL
        response = requests.get(submission_url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')

            # Find the element containing the source code
            source_elem = soup.select('#program-source-text')

            # Check if the element was found
            if source_elem:
                # Convert the HTML source code to plain text
                source = html.unescape(source_elem[0].decode_contents())

                # Path to the successful_submission file
                successful_submissions = "top_friends_successful_submissions.csv"

                # Open the file in append mode and write the data
                with open(successful_submissions, "a", newline="", encoding='utf-8') as file:
                    writer = csv.DictWriter(file, fieldnames=["#", "Number", 'Source_code'])

                    # Write the header row if the file is empty
                    if file.tell() == 0:
                        writer.writeheader()

                    # Append the data rows
                    writer.writerows([{"#": submission, "Number": num, "Source_code":source}])
            else:
                print(f'Failed to find source code for submission {submission} in contest {num}')
                                                
                # Path to the failed_submission.csv file
                failed_submission = "top_friends_failed_submission.csv"

                # Open the file in append mode and write the data
                with open(failed_submission, "w", newline="", encoding='utf-8') as file:
                    writer = csv.DictWriter(file, fieldnames=["#", "Number"])

                    # Write the header row if the file is empty
                    if file.tell() == 0:
                        writer.writeheader()

                    # Append the data rows
                    writer.writerows([{"#": submission, "Number": num}])
                    
                winsound.PlaySound('frog.wav', winsound.SND_FILENAME)
                break
                
        else:
            print(f'Failed to retrieve submission {submission} in contest {num}')
        time.sleep(2)
        